In [1]:
import argparse
import logging
import torch

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
)


In [5]:
model_dir='/oper/ch/git/DeepSpeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/output_chinese_bloom_560m_5data/'
device = torch.device("cuda:0")
config = AutoConfig.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir,fast_tokenizer=True)

In [6]:
def get_model(config, model_path, tokenizer):

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        from_tf=bool(".ckpt" in model_path),
        config=config,
    )
    model.resize_token_embeddings(len(tokenizer))

    # prepare the tokenizer and model config
    tokenizer.pad_token = tokenizer.eos_token
    model.config.end_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = model.config.eos_token_id

    return model
model_fintuned = get_model(config,model_dir,tokenizer)

In [7]:
model_fintuned.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250680, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (1): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementw

In [9]:
def generate(model,
             tokenizer,
             inputs,
             num_beams=1,
             num_beam_groups=1,
             do_sample=False,
             num_return_sequences=1,
             max_new_tokens=100):

    generate_ids = model.generate(inputs.input_ids,
                                  num_beams=num_beams,
                                  num_beam_groups=num_beam_groups,
                                  do_sample=do_sample,
                                  num_return_sequences=num_return_sequences,
                                  max_new_tokens=max_new_tokens)

    result = tokenizer.batch_decode(generate_ids,
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)
    return result

def generate_constrastive_search(model,
                                 tokenizer,
                                 inputs,
                                 top_k=4,
                                 penalty_alpha=0.6,
                                 num_return_sequences=1,
                                 max_new_tokens=100):

    generate_ids = model.generate(inputs.input_ids,
                                  top_k=top_k,
                                  penalty_alpha=penalty_alpha,
                                  num_return_sequences=num_return_sequences,
                                  max_new_tokens=max_new_tokens)

    result = tokenizer.batch_decode(generate_ids,
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)
    return result


In [20]:
prompts = [
    "Human: 中国有多少人 Assistant:",
    "Human: 用几句话向6岁的孩子解释登月。 Assistant:",
    "Human: 为什么无法制成永动机。 Assistant:",
    "Human: 最伟大的美国总统? Assistant:", "Human: 望远镜是如何工作的? Assistant:",
    "Human: 鸟类为什么要南迁过冬? Assistant:",
    "Human: 怎么说服男朋友买烤箱？ Assistant:",
    ]
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    print("==========finetune: Greedy=========")
    r_finetune_g = generate(model_fintuned,
                            tokenizer,
                            inputs,
                            num_beams=1,
                            num_return_sequences=1,
                            max_new_tokens=100)
    print(r_finetune_g)

==========finetune: Greedy=========
['Human: 中国有多少人 Assistant: 谢邀。 这个问题其实很难回答，因为每个人对中国的定义都不一样。我个人认为，中国有上亿人，但实际上，只有不到1.5亿人。 事实上，中国人口基数大，但人口结构却相对复杂。中国人口结构复杂主要体现在： 1.中国人口结构复杂，人口比例复杂，人口结构复杂。 2.中国人口结构复杂，人口比例复杂，人口结构复杂。 3.中国人口结构复杂，人口比例']
==========finetune: Greedy=========
['Human: 用几句话向6岁的孩子解释登月。 Assistant: 登月，是人类历史上最伟大的成就之一，也是人类历史上最伟大的梦想之一。 登月，是人类历史上最伟大的成就之一，也是人类历史上最伟大的梦想之一。 登月，是人类历史上最伟大的成就之一，也是人类历史上最伟大的梦想之一。 登月，是人类历史上最伟大的成就之一，也是人类历史上最伟大的梦想之一。 登月，是人类历史上最伟大的成就之一，也是人类历史上最伟大的梦想之一。']
==========finetune: Greedy=========
['Human: 为什么无法制成永动机。 Assistant: 人类无法制造永动机，因为永动机需要一个无限大空间来描述，而这个空间是有限的，所以无法制造。<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|']
==========finetune: Greedy=========
['Human: 最伟大的美国总统? Assistant: 杰弗逊·打电话给具象化，并带入现实，并最终将现实与具象化联系在一起。 杰弗逊在打电话给具象化时，首先将现实中的自己带入具象化，并最终将现实中的自己与具象化联系在一起。 杰弗逊在打电话给具象化时，首先将现实中的自己带入具象化，并最终将现实中的自己与具象化联系在一起。 ']
==========finetune: Greedy=========
['Human: 望远镜是如何工作的? Assistant: 谢邀。